## Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
path_data='./data/results/'
name_file_lieux_equipements_culturels = 'nbre_equipement_parcommuneetcodepostal_ET_LatitudeLongitude_ET_POPULATION.csv'

## Load data

In [3]:
# latitude, longitude, Population
df_synthese=pd.read_csv(path_data+name_file_lieux_equipements_culturels, sep='\t', on_bad_lines='skip' ,encoding='utf-8',low_memory=False)
df_synthese.head()

,Commune,PTOT,REG,DEP,Région,latitude,longitude,code_insee,Type équipement ou lieu,count,categorie
0,L'Abergement-Clémenciat,848.0,84.0,1.0,Auvergne-Rhône-Alpes,46.153426,4.926114,1001.0,Monument,1,patrimoine
1,Ambérieu-en-Bugey,15240.0,84.0,1.0,Auvergne-Rhône-Alpes,45.960848,5.372926,1004.0,Bibliothèque,1,vivant
2,Ambérieu-en-Bugey,15240.0,84.0,1.0,Auvergne-Rhône-Alpes,45.960848,5.372926,1004.0,Cinéma,1,vivant
3,Ambérieu-en-Bugey,15240.0,84.0,1.0,Auvergne-Rhône-Alpes,45.960848,5.372926,1004.0,Monument,3,patrimoine
4,Ambérieu-en-Bugey,15240.0,84.0,1.0,Auvergne-Rhône-Alpes,45.960848,5.372926,1004.0,Service d'archives,1,patrimoine


## Visu pour une commune
### 1-sunburst

In [4]:
nom_commune = 'Ambérieu-en-Bugey'
series = df_synthese.loc[df_synthese['Commune']==nom_commune,['Type équipement ou lieu','categorie']]
series

,Type équipement ou lieu,categorie
1,Bibliothèque,vivant
2,Cinéma,vivant
3,Monument,patrimoine
4,Service d'archives,patrimoine


In [5]:
df = series.reset_index()
df.head()

,index,Type équipement ou lieu,categorie
0,1,Bibliothèque,vivant
1,2,Cinéma,vivant
2,3,Monument,patrimoine
3,4,Service d'archives,patrimoine


In [6]:
fig = px.sunburst(df, path=['categorie','Type équipement ou lieu' ], values='index')

# Affichage du graphique
fig.show()

### 2-indicateur d'accès à la culture

In [7]:
Nbre_habitants = 1000# nbre de habitant par commune pour le calcul du score
seuil = 10

In [8]:
df_synthese.head(10)

,Commune,PTOT,REG,DEP,Région,latitude,longitude,code_insee,Type équipement ou lieu,count,categorie
0,L'Abergement-Clémenciat,848.0,84.0,1.0,Auvergne-Rhône-Alpes,46.153426,4.926114,1001.0,Monument,1,patrimoine
1,Ambérieu-en-Bugey,15240.0,84.0,1.0,Auvergne-Rhône-Alpes,45.960848,5.372926,1004.0,Bibliothèque,1,vivant
2,Ambérieu-en-Bugey,15240.0,84.0,1.0,Auvergne-Rhône-Alpes,45.960848,5.372926,1004.0,Cinéma,1,vivant
3,Ambérieu-en-Bugey,15240.0,84.0,1.0,Auvergne-Rhône-Alpes,45.960848,5.372926,1004.0,Monument,3,patrimoine
4,Ambérieu-en-Bugey,15240.0,84.0,1.0,Auvergne-Rhône-Alpes,45.960848,5.372926,1004.0,Service d'archives,1,patrimoine
5,Ambérieux-en-Dombes,1921.0,84.0,1.0,Auvergne-Rhône-Alpes,45.996180,4.912273,1005.0,Bibliothèque,1,vivant
6,Ambérieux-en-Dombes,1921.0,84.0,1.0,Auvergne-Rhône-Alpes,45.996180,4.912273,1005.0,Monument,1,patrimoine
7,Ambronay,2951.0,84.0,1.0,Auvergne-Rhône-Alpes,46.005591,5.357607,1007.0,Bibliothèque,1,vivant
8,Ambronay,2951.0,84.0,1.0,Auvergne-Rhône-Alpes,46.005591,5.357607,1007.0,Centre culturel,1,vivant
9,Ambronay,2951.0,84.0,1.0,Auvergne-Rhône-Alpes,46.005591,5.357607,1007.0,Monument,2,patrimoine


In [9]:
# somme du nbre de culturel par commune ((nbre patrimoine+nbre vivant)=sum(count par commune))
nbre_culturel = df_synthese.groupby(['code_insee','Commune','Région','latitude','longitude','PTOT','REG','DEP'])['count'].sum() # create Série avec MultiIndex
# Convertir la Série avec MultiIndex en DataFrame
df_nbre_culturel = nbre_culturel.reset_index(name='nbr_total_culturel')

# Calcul du score par commune = (nbre patrimoine+ nbre vivant)/population
# score par commune pour 50 habitants = score par commune * 50
df_nbre_culturel['Nbre_culturels_N_habitants'] = (df_nbre_culturel['nbr_total_culturel'] / df_nbre_culturel['PTOT'])*Nbre_habitants


# seuillage du score, nécessaire pour l'affichage: si le score>1, score=1
df_nbre_culturel['Nbre_culturels_N_habitants_seuillee'] = df_nbre_culturel['Nbre_culturels_N_habitants'] 
df_nbre_culturel.loc[df_nbre_culturel['Nbre_culturels_N_habitants_seuillee']>seuil,'Nbre_culturels_N_habitants_seuillee']=seuil


# suprimer les score=Inf dues à une population=0
# Remplacer les valeurs Inf par NaN
df_nbre_culturel.replace([np.inf, -np.inf], np.nan, inplace=True)
# Supprimer les lignes contenant des NaN
df_nbre_culturel = df_nbre_culturel.dropna()

In [10]:
# rename column for visu
df_nbre_culturel = df_nbre_culturel.rename(columns={'Nbre_culturels_N_habitants_seuillee':'Nbre culturels pour'+str(Nbre_habitants)+' habitants'})
df_nbre_culturel.head()

fig = px.scatter_map(df_nbre_culturel, lat="latitude", lon="longitude", color='Nbre culturels pour'+str(Nbre_habitants)+' habitants', size="nbr_total_culturel",
                 # color_continuous_scale=px.colors.cyclical.IceFire,
                  color_discrete_sequence=px.colors.qualitative.Pastel,
                  #px.colors.cyclical.IceFire, #'PuBu',#
                    size_max=15, zoom=10,
                  map_style="carto-positron")
fig.show()

In [11]:
px.line(df_nbre_culturel['Nbre_culturels_N_habitants'] )

In [12]:
px.line(df_nbre_culturel['Nbre culturels pour'+str(Nbre_habitants)+' habitants'])